In [1]:
import sys; sys.path.append('./')
from utils import Utils
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
import time
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import qgrid

In [2]:
ut = Utils()
X, Y = ut.get_matrixes()

In [3]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [4]:
def select_features(modelo, n_features, fwd, fltg):
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=2,
           scoring='accuracy',
           cv=0,
           n_jobs=-1,
           pre_dispatch=10)
    
    return sfs

In [5]:
def trainSFS(n_features,clf):
    
    forward = True
    floating = False
   
        
        
    tiempo_i = time.time()
    
    #Implemetamos la metodología de validación 

    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, Y, random_state=0, stratify=Y, shuffle=True)
    for train, test in skf.split(X_train_d,y_train_d):
        

        X_train, X_test = X[train], X[train]
        y_train, y_test = Y[train], Y[train]  
        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        sf = select_features(clf, n_features, forward, floating)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        print("iteracion: " + str(j))
        print("feature: " + str(n_features))
        sf = sf.fit(X_train, y_train)

        Errores[j] = 1-sf.k_score_
        j+=1
    return  str(np.mean(Errores)), str(np.std(Errores)), str(sf.k_score_*100), str(time.time()-tiempo_i),sf.k_feature_idx_

In [6]:
def train(clf):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación

    j = 0
    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)

    for train, test in skf.split(X,Y):

        Xtrain, Ytrain, Xtest, Ytest = ut.get_training_test(X=X, Y=Y, train=train, test=test)

        #Aquí se entran y se valida el modelo sin hacer selección de características

        ######

        # Entrenamiento el modelo.
        model = clf.fit(Xtrain,Ytrain)

        # Validación del modelo
        ypred = model.predict(Xtest)

        #######

        Errores[j] = classification_error(ypred, Ytest)
        j+=1
    
    m_errores = np.mean(Errores)
    ic_errores = np.std(Errores)
    eficiencia = (1 - np.mean(Errores))*100
    tEjec = time.time()-tiempo_i
                        
    print('finalizado clf sin selección',str(m_errores), str(ic_errores), str(eficiencia), str(tEjec))
    return  m_errores, str(np.std(Errores)), str((1 - np.mean(Errores))*100), str(time.time()-tiempo_i)

In [8]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin Selección','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SFS']),
    '# de características seleccionadas' : pd.Series([23, 3, 5, 10,15,20]),
   })
ic_ = []
tEjec_ = []
errorVal_ = []
eficiencia_ = []
caracteristicas=[]
model = OneVsRestClassifier(SVC(C=100,kernel='rbf', gamma=0.10, probability=True), n_jobs=-1)
for e, i in zip(df_types['Tecnica'], df_types['# de características seleccionadas']):
    
    if(e == "SVM sin Selección"):
        errorVal, ic, eficiencia, tEjec = train(model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
    else:
        print('entrenando: ',e,i)
        errorVal, ic, eficiencia, tEjec,car = trainSFS(i, model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
        caracteristicas.append(car)



finalizado clf sin selección 0.08781244282505538 0.026856402410310624 91.21875571749446 10.581037282943726
entrenando:  SVM + SFS 3
iteracion: 0
feature: 3


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   33.2s finished

[2021-01-16 23:42:01] Features: 1/3 -- score: 0.7754297994269341[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   21.2s finished

[2021-01-16 23:42:23] Features: 2/3 -- score: 0.8481375358166189[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   16.2s finished

[2021-01-16 23:42:39] Features: 3/3 -- score: 0.8825214899713467[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


iteracion: 1
feature: 3


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   18.3s finished

[2021-01-16 23:42:57] Features: 1/3 -- score: 0.75[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   17.0s finished

[2021-01-16 23:43:14] Features: 2/3 -- score: 0.8137535816618912[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   16.7s finished

[2021-01-16 23:43:31] Features: 3/3 -- score: 0.839541547277937[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


iteracion: 2
feature: 3


...

[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   27.8s finished

[2021-01-17 00:17:06] Features: 1/20 -- score: 0.75[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   22.5s finished

[2021-01-17 00:17:29] Features: 2/20 -- score: 0.8137535816618912[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   16.1s finished

[2021-01-17 00:17:45] Features: 3/20 -- score: 0.839541547277937[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   12.9s finished

[2021-01-17 00:17:58] Features: 4/20 -- score: 0.8674785100286533[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   11.2s finished

[2021-01-17 00:18:09] Features: 5/20 -- score: 0.8904011461318052[Parallel(n_jobs=-1)]: Usi

iteracion: 2
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   31.6s finished

[2021-01-17 00:20:38] Features: 1/20 -- score: 0.744269340974212[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   20.7s finished

[2021-01-17 00:20:59] Features: 2/20 -- score: 0.8166189111747851[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   10.7s finished

[2021-01-17 00:21:10] Features: 3/20 -- score: 0.8599570200573066[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   13.8s finished

[2021-01-17 00:21:23] Features: 4/20 -- score: 0.8878939828080229[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   14.2s finished

[2021-01-17 00:21:38] Features: 5/20 -- score: 0.9079512893982808[Parallel(n_

iteracion: 3
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   30.3s finished

[2021-01-17 00:23:55] Features: 1/20 -- score: 0.7769423558897243[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   13.6s finished

[2021-01-17 00:24:08] Features: 2/20 -- score: 0.8070175438596491[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   12.6s finished

[2021-01-17 00:24:21] Features: 3/20 -- score: 0.8539205155746509[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   17.3s finished

[2021-01-17 00:24:38] Features: 4/20 -- score: 0.8800572860723237[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   12.9s finished

[2021-01-17 00:24:51] Features: 5/20 -- score: 0.8933046902971715[Parallel(n

In [12]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin Selección','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SFS']),
    '# de características seleccionadas' : pd.Series([23, 3, 5, 10,15,20]),
   })
df_types["Eficiencia(%)"] = eficiencia_
df_types["Error de validación"] = errorVal_
df_types["IC(std)"] = ic_
df_types["Tiempo de ejecución"] = tEjec_

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)
df_types.to_csv('car_select_SVM.csv')
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

/home/juan/simulation/simulation/lib/python3.8/site-packages/qgrid/grid.py:1329: FutureWarning: inplace is deprecated and will be removed in a future version.
  df.index.set_levels(col_series, level=key_index, inplace=True)


Eficiencia(%)  \
Tecnica           # de características seleccionadas                      
SVM sin Selección 23                                  91.21875571749446   
SVM + SFS         3                                   85.39205155746508   
                  5                                   89.33046902971715   
                  10                                  90.47619047619048   
                  15                                  91.83673469387756   
                  20                                  92.19477264590047   

                                                      Error de validación  \
Tecnica           # de características seleccionadas                        
SVM sin Selección 23                                             0.087812   
SVM + SFS         3                                   0.14101485677968972   
                  5                                   0.09965018845722573   
                  10                                  0.08201152954018284   
                  15                                  0.06920830653178175   
                  20                                  0.06732825463166717   

                                                                   IC(std)  \
Tecnica           # de características seleccionadas                         
SVM sin Selección 23                                  0.026856402410310624   
SVM + SFS         3                                   0.015480697976276518   
                  5                                    0.00858206861757379   
                  10                                  0.012910089211737327   
                  15                                  0.012820500489625524   
                  20                                    0.0125860147118901   

                                                       Tiempo de ejecución  
Tecnica           # de características seleccionadas                        
SVM sin Selección 23                                  10.583084106445312 s  
SVM + SFS         3                                   254.23147821426392 s  
                  5                                    353.3728377819061 s  
                  10                                   568.2071440219879 s  
                  15                                   728.5093853473663 s  
                  20                                   813.4110102653503 s

In [7]:
model = OneVsRestClassifier(SVC(C=100,kernel='rbf', gamma=0.10, probability=True), n_jobs=-1)

In [8]:
_, _, _, _,car = trainSFS(20, model)

iteracion: 0
feature: 20


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   28.4s finished

[2021-01-19 00:29:40] Features: 1/20 -- score: 0.7754297994269341[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   26.3s finished

[2021-01-19 00:30:06] Features: 2/20 -- score: 0.8481375358166189[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   14.6s finished

[2021-01-19 00:30:21] Features: 3/20 -- score: 0.8825214899713467[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    7.6s finished

[2021-01-19 00:30:28] Features: 4/20 -- score: 0.8954154727793696[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    7.2s finished



iteracion: 1
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   25.8s finished

[2021-01-19 00:32:49] Features: 1/20 -- score: 0.75[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   19.5s finished

[2021-01-19 00:33:08] Features: 2/20 -- score: 0.8137535816618912[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   12.9s finished

[2021-01-19 00:33:21] Features: 3/20 -- score: 0.839541547277937[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   12.1s finished

[2021-01-19 00:33:33] Features: 4/20 -- score: 0.8674785100286533[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   10.7s finished

[2021-01-19 00:33:44] Features: 5/20 -- score: 0.8904011461318052[Parallel(n_jobs=-1)]: Usi

iteracion: 2
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   32.2s finished

[2021-01-19 00:36:07] Features: 1/20 -- score: 0.744269340974212[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   19.2s finished

[2021-01-19 00:36:26] Features: 2/20 -- score: 0.8166189111747851[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   20.3s finished

[2021-01-19 00:36:46] Features: 3/20 -- score: 0.8599570200573066[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   15.9s finished

[2021-01-19 00:37:02] Features: 4/20 -- score: 0.8878939828080229[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   15.0s finished

[2021-01-19 00:37:17] Features: 5/20 -- score: 0.9079512893982808[Parallel(n_

iteracion: 3
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   17.7s finished

[2021-01-19 00:39:27] Features: 1/20 -- score: 0.7769423558897243[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   22.4s finished

[2021-01-19 00:39:49] Features: 2/20 -- score: 0.8070175438596491[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   18.7s finished

[2021-01-19 00:40:08] Features: 3/20 -- score: 0.8539205155746509[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   17.0s finished

[2021-01-19 00:40:25] Features: 4/20 -- score: 0.8800572860723237[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   14.9s finished

[2021-01-19 00:40:40] Features: 5/20 -- score: 0.8933046902971715[Parallel(n

In [9]:
car

(0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21)

In [ ]:
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 21)